In [1]:
import numpy  as np 
import pandas as pd 
import os
import cv2 
import gc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from tensorflow import set_random_seed

import keras
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.applications import DenseNet121
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.activations import softmax, relu
from keras.optimizers import Adam

# Global constants
IMG_DIM      = 256
BATCH_SIZE   = 32
CHANNEL_SIZE = 3
NUM_CLASSES  = 5

print(os.listdir("../"))
print(os.listdir("../input/"))
print(os.listdir("../input/aptos2019-blindness-detection"))
print(os.listdir("../input/densenetmulti"))

INPUT_FOLDER     ='../input/aptos2019-blindness-detection/'
TEST_IMAGES_DIR  = INPUT_FOLDER + "test_images/"

ModuleNotFoundError: No module named 'keras'

# Get in test data

In [ ]:
test_df = pd.read_csv(INPUT_FOLDER + 'test.csv')
test_df['id_code'] = test_df['id_code'].apply(lambda x:  x + '.png')

# Preprocessing
We need to crop and preprocess all the test images into the working directory

In [ ]:
def label_convert(y_val):
    y_val = y_val.astype(int).sum(axis=1) - 1
    #y_val= np.argmax(y_val, axis=1)
    return y_val

def crop(bgr):
    
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    
    thresh = 5
    
    # remove any black regions, where black is y under the threshold
    rowMaxes = gray.max(axis=1)
    top = 0
    while rowMaxes[top] < thresh:
        top += 1
    bottom = len(rowMaxes) - 1
    while rowMaxes[bottom] < thresh:
        bottom -= 1
        
    middleRow = gray[int((bottom-top)/2)]
    left = 0
    while middleRow[left] < thresh:
        left += 1
    right = len(middleRow) - 1
    while middleRow[right] < thresh:
        right -= 1
        
    height = bottom - top
    width  = right - left
        
    if height < 100 or width < 100:
        print("Error: squareUp: bottom:", bottom, "top:", top)
        print("Error: squareUp: right:", right, "left:", left)
        return bgr
    
    return bgr[top:bottom, left:right]

# assumes BGR
def colourfulEyes(bgr, weight=4, gamma=15):
        
    # convert to y, cr, cb so that we can modify the image based on just the y (brightness)
    ycc = cv2.cvtColor(bgr, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycc)

    # perform bens algorithm on the y component
    y = cv2.addWeighted(y, weight, cv2.GaussianBlur(y, (0,0), gamma), -weight, 128)

    # merge the ycc back together, and recolor it
    ycc_modified = cv2.merge((y, cr, cb))
    modified = cv2.cvtColor(ycc_modified, cv2.COLOR_YCrCb2BGR)
    
    return modified 

def processImageBgrToRgb(bgr):
    modified = crop(bgr) 
    modified = cv2.resize(modified, (IMG_DIM, IMG_DIM))
    modified = colourfulEyes(modified)
    modified = cv2.cvtColor(modified, cv2.COLOR_BGR2RGB)
    return modified

# Import the model

In [ ]:
def create_model(img_dim, CHANNELS, n_class):
    
    input_tensor=Input(shape=(img_dim, img_dim, CHANNELS))
    
    base_model = DenseNet121(weights=None, include_top=False, input_tensor=input_tensor)
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.2)(x)
    x = Dense(512, activation=relu)(x)
    x = Dropout(0.15)(x)
    x = BatchNormalization()(x)
    output_layer = Dense(n_class, activation='sigmoid', name="Output_Layer")(x)
    model = Model(input_tensor, output_layer)
    
    return model

model=create_model(IMG_DIM, CHANNEL_SIZE, NUM_CLASSES)
model.load_weights("../input/densenetmulti/dense-multi-0.8929.h5")
model.compile(optimizer=Adam(lr=0.00005), loss=keras.losses.mean_squared_error,  metrics=['accuracy'])
gc.collect()

# Train set pipeline test
Comment out before submission

In [ ]:
# train_df = pd.read_csv(INPUT_FOLDER + 'train.csv')
# train_df['id_code'] = train_df['id_code'].apply(lambda x:  x + '.png')

# block_size = 500
# total = train_df.index.size

# # have to do everything in blocks of images
# for start in range(0, test_df.index.size, block_size):
    
#     gc.collect()
    
#     # get subset
#     end = start + block_size
#     end   = start + block_size
#     if end > total:
#         end = total
    
#     # process images
#     img_list = np.empty((end-start, IMG_DIM, IMG_DIM, 3))
#     for i, filename in enumerate(train_df[start:end].id_code):
#         try:
#             bgr = cv2.imread(INPUT_FOLDER + "train_images/" + filename)
#             img_list[i,:,:,:] = processImageBgrToRgb(bgr)
#         except:
#             img_list[i,:,:,:] = 128.
    
#     # re scale to between 0 and 1
#     img_list *= 1./255.
    
#     # make predictions
#     predictions = model.predict(img_list, batch_size=BATCH_SIZE, verbose=1)
#     predictions = predictions > 0.5
#     y_pred = label_convert(predictions)
#     y_val  = train_df['diagnosis'][start:end]
    
#     print(confusion_matrix(y_val, y_pred))
    
#     val_kappa = cohen_kappa_score(y_val, y_pred, weights='quadratic')

#     print(f"{start} - {end}: kappa: {val_kappa}")


In [ ]:

# figure=plt.figure(figsize=(22,20))
# for target_class in (train_df['diagnosis'].unique()):
#     sample = train_df[train_df.diagnosis == target_class].sample(2)
#     for i, filename in enumerate(sample.id_code):
        
#         # original           
#         bgr = cv2.imread(INPUT_FOLDER + "train_images/" + filename)
#         rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
#         ax = figure.add_subplot(5,4, int(target_class)*4+2*i+1)
#         plt.imshow(rgb)
       
#         # modified
#         modified = processImageBgrToRgb(bgr)
#         ax = figure.add_subplot(5,4, int(target_class)*4+2*i+2)
#         plt.imshow(modified)

# Perform predictions in slices 

In [ ]:
# test_df = test_df_extended
block_size = 500
total      = test_df.index.size
y_pred_list = np.zeros(total, dtype=np.int)

# have to do everything in blocks of images
for start in range(0, total, block_size):
    
    gc.collect()
    
    # get subset
    end = start + block_size
    if end > total:
        end = total
    
    # process images
    img_list = np.empty((end-start, IMG_DIM, IMG_DIM, 3))
    for i, filename in enumerate(test_df[start:end].id_code):
        try:
            bgr = cv2.imread(TEST_IMAGES_DIR + filename)
            img_list[i,:,:,:] = processImageBgrToRgb(bgr)
        except:
            img_list[i,:,:,:] = 128.
    
    # re scale to between 0 and 1
    img_list *= 1./255.
    
    # make predictions
    predictions = model.predict(img_list, batch_size=BATCH_SIZE, verbose=1)
    predictions = predictions > 0.5
    y_pred_list[start:end] = label_convert(predictions)

    print(f"{start} - {end} finished")

# save to csv
test_df['diagnosis'] = y_pred_list
test_df.to_csv('submission.csv', index=False)

In [ ]:
# print(test_df)